In [40]:
pip install pandas==2.0.3


  Using cached pandas-2.0.3-cp310-cp310-win_amd64.whl.metadata (18 kB)
Using cached pandas-2.0.3-cp310-cp310-win_amd64.whl (10.7 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
  Rolling back uninstall of pandas
  Moving to c:\users\phuc.nguyentuan\appdata\local\miniconda3\envs\myenv\lib\site-packages\pandas-1.5.3.dist-info\
   from C:\Users\phuc.nguyentuan\AppData\Local\miniconda3\envs\myenv\Lib\site-packages\~andas-1.5.3.dist-info
  Moving to c:\users\phuc.nguyentuan\appdata\local\miniconda3\envs\myenv\lib\site-packages\pandas\__init__.py
   from C:\Users\phuc.nguyentuan\AppData\Local\Temp\pip-uninstall-v8dduhb6\__init__.py
  Moving to c:\users\phuc.nguyentuan\appdata\local\miniconda3\envs\myenv\lib\site-packages\pandas\__pycache__\__init__.cpython-310.pyc
   from C:\Users\phuc.nguyentuan\AppData\Local\Temp\pip-uninstall-v8dduhb6\__pycache__\__init__.cpython-310.pyc
  Moving t

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\phuc.nguyentuan\\AppData\\Local\\miniconda3\\envs\\myenv\\Lib\\site-packages\\pandas\\_libs\\algos.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [1]:
import pandas as pd
import os
import sys
from pathlib import Path

from f00_read_configs import read_excel_data_from_row
from f00_read_configs import set_config_df_types

In [2]:
base_path = Path(r'C:\Users\phuc.nguyentuan\Python_project\New Folder\Tien')
filepath = 'read_workflow_config_test.xlsx'
sheet_name = 'F001_(4)'  
config_df = read_excel_data_from_row(filepath, sheet_name)
config_df = set_config_df_types(config_df)
config_df

,input_folder_path,input_file_name,input_file_type,input_sheet_name,output_folder_path,output_file_name,output_sheet_name,base_input_class,linked_input_class,remove_rows_list,applied_column,criteria_to_remove_row,criteria_value,Num
0,C:\Users\phuc.nguyentuan\Python_project\New fo...,Sample_for_test_process_data_Q1&Q2.xlsx,.xlsx,MG,output,Test_df_remove,Sheet1,a,a,"POsition, year_month",NaN,NaN,NaN,1
1,input,Sample_for_test_process_data_Q3&Q4,.xlsx,MG,C:\Users\phuc.nguyentuan\Python_project\New fo...,Test_df_remove.xlsx,Sheet2,b,b,YEAR_month,year_month,=,Total,2
2,nan,nan,nan,NaN,nan,nan,NaN,NaN,a,NaN,year_month,=,Applied filters:,3
3,nan,nan,nan,NaN,nan,nan,NaN,NaN,a,NaN,position,contain,M,4
4,nan,nan,nan,NaN,nan,nan,NaN,NaN,a,NaN,year_month,=,Total,5


In [6]:
from f00_read_configs import read_config_except_columns
base_path = Path(r'C:\Users\phuc.nguyentuan\Python_project\New Folder\Tien')
config_file = 'read_workflow_config_test.xlsx'
sheet_name = 'F001_(4)'
config_file_path = base_path / config_file
file_config_df, cal_df, mapping_zone_df = read_config_except_columns(base_path, config_file_path, sheet_name, lower_case_except_file_zone=True,
                                                       execpt_cal_df_columns={"criteria_value"}, 
                                                       execpt_mapping_zone_df_columns=None)

C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\f00_read_configs.py:254: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  file_config_df = file_config_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [5]:
cal_df

,linked_input_class,remove_rows_list,applied_column,criteria_to_remove_row,criteria_value,Num
0,a,"position, year_month",NaN,NaN,NaN,1
1,b,year_month,year_month,=,Total,2
2,a,NaN,year_month,=,Applied filters:,3
3,a,NaN,position,contain,M,4
4,a,NaN,year_month,=,Total,5


In [7]:
import pandas as pd
import numpy as np
from pathlib import Path

def read_input_file_type(input_file_path, input_file_type, sheet_name=None):
    if input_file_type == '.xlsx':
        return pd.read_excel(input_file_path, sheet_name=sheet_name)
    elif input_file_type == '.csv':
        return pd.read_csv(input_file_path)
    elif input_file_type == '.txt':
        return pd.read_csv(input_file_path, delimiter='\t')
    else:
        raise ValueError(f"Unsupported file type: {input_file_type}")
        
def apply_removal_criteria(df, criteria):
    
    column = criteria['applied_column']
    if column not in df.columns:
        print(f"The column '{column}' does not exist in the dataframe. Skipping the removal criteria.")
        return df
    
    # column = criteria['applied_column']
    operation = criteria['criteria_to_remove_row']
    value = criteria['criteria_value']
    
    if operation == '=':
        # Remove rows where the column value equals the criteria value
        df = df[df[column].astype(str).str.strip() != value]
    elif operation == '> =':
        # Remove rows where the column value is less than the criteria value
        df = df[df[column] < float(value)]
    elif operation == '>':
        # Remove rows where the column value is less than or equal to the criteria value
        df = df[df[column] <= float(value)]
    elif operation == '< =':
        # Remove rows where the column value is greater than the criteria value
        df = df[df[column] > float(value)]
    elif operation == '<':
        # The '<' case is correctly implemented in the original prompt as removing rows where the value is greater or equal
        df = df[df[column] >= float(value)]  # Correcting based on the initial explanation
    elif operation == 'contain':
        # Remove rows that contain the criteria value
        df = df[~df[column].str.contains(value, na=False, regex=True)]

    return df

def run_removal_automation(file_config_df, cal_df, base_path):
    base_path = Path(base_path)  # Ensure base_path is a Path object

    for _, file_info in file_config_df.iterrows():
        if pd.isna(file_info['input_folder_path']) or pd.isna(file_info['input_file_name']):
            continue

        input_file_name = file_info['input_file_name']
        if not input_file_name.lower().endswith(file_info['input_file_type']):
            input_file_name += file_info['input_file_type']

        output_file_name = file_info['output_file_name']
        if not output_file_name.lower().endswith('.xlsx'):
            output_file_name += '.xlsx'

        input_file = base_path / file_info['input_folder_path'] / input_file_name
        output_file = base_path / file_info['output_folder_path'] / output_file_name

        if not input_file.is_file():
            print(f"File not found: {input_file}")
            continue

        df = read_input_file_type(input_file, file_info['input_file_type'], file_info.get('input_sheet_name'))

        criteria_rows = cal_df[cal_df['linked_input_class'] == file_info['base_input_class']]
        if criteria_rows.empty:
            print(f"No matching criteria found for key: {file_info['base_input_class']}")
            continue

        for idx, criteria in criteria_rows.iterrows():
            print(f"Applying criteria: {criteria['applied_column']} {criteria['criteria_to_remove_row']} {criteria['criteria_value']}")
            criteria_dict = {
                'applied_column': criteria['applied_column'].strip().lower(),
                'criteria_to_remove_row': criteria['criteria_to_remove_row'],
                'criteria_value': criteria['criteria_value']
            }
            df = apply_removal_criteria(df, criteria_dict)

        # Prepare to save to Excel with specific sheet name
        output_file.parent.mkdir(parents=True, exist_ok=True)
        sheet_name = file_info.get('output_sheet_name', 'Sheet1')  # Default to 'Sheet1' if not specified
        with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"Data written to {sheet_name} in {output_file}")

In [5]:
run_removal_automation(file_config_df, cal_df, base_path)

Header set successfully.
Applying criteria: year_month = nan
Applying criteria: year_month = Applied filters:
Applying criteria: position contain M
Applying criteria: year_month = Total


NameError: name 'output_file_path' is not defined

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

def read_input_file_type(input_file_path, input_file_type, sheet_name=None):
    if input_file_type == '.xlsx':
        return pd.read_excel(input_file_path, sheet_name=sheet_name)
    elif input_file_type == '.csv':
        return pd.read_csv(input_file_path)
    elif input_file_type == '.txt':
        return pd.read_csv(input_file_path, delimiter='\t')
    else:
        raise ValueError(f"Unsupported file type: {input_file_type}")
        
def apply_removal_criteria(df, criteria):
    
    column = criteria['applied_column']
    if column not in df.columns:
        print(f"The column '{column}' does not exist in the dataframe. Skipping the removal criteria.")
        return df
    
    # column = criteria['applied_column']
    operation = criteria['criteria_to_remove_row']
    value = criteria['criteria_value']
    
    if operation == '=':
        # Remove rows where the column value equals the criteria value
        df = df[df[column].astype(str).str.strip() != value]
    elif operation == '> =':
        # Remove rows where the column value is less than the criteria value
        df = df[df[column] < float(value)]
    elif operation == '>':
        # Remove rows where the column value is less than or equal to the criteria value
        df = df[df[column] <= float(value)]
    elif operation == '< =':
        # Remove rows where the column value is greater than the criteria value
        df = df[df[column] > float(value)]
    elif operation == '<':
        # The '<' case is correctly implemented in the original prompt as removing rows where the value is greater or equal
        df = df[df[column] >= float(value)]  # Correcting based on the initial explanation
    elif operation == 'contain':
        # Remove rows that contain the criteria value
        df = df[~df[column].str.contains(value, na=False, regex=True)]

    return df

def run_removal_automation(file_config_df, cal_df, base_path):
    base_path = Path(base_path)
    excel_writers = {}  # Dictionary to manage ExcelWriter objects for each output file

    for _, file_info in file_config_df.iterrows():
        if pd.isna(file_info['input_folder_path']) or pd.isna(file_info['input_file_name']):
            continue

        input_file_path = base_path / file_info['input_folder_path'] / file_info['input_file_name']
        if not input_file_path.suffix:
            input_file_path = input_file_path.with_suffix(file_info['input_file_type'])

        output_file_path = base_path / file_info['output_folder_path'] / file_info['output_file_name']
        if not output_file_path.suffix:
            output_file_path = output_file_path.with_suffix('.xlsx')

        sheet_name = file_info.get('output_sheet_name', 'Sheet1')

        if not input_file_path.is_file():
            print(f"File not found: {input_file_path}")
            continue

        df = read_input_file_type(input_file_path, file_info['input_file_type'], file_info.get('input_sheet_name'))

        criteria_rows = cal_df[cal_df['linked_input_class'] == file_info['base_input_class']]
        if criteria_rows.empty:
            print(f"No matching criteria found for key: {file_info['base_input_class']}")
            continue

        # Header identification and setting
        header_row_index = None
        header_values = criteria_rows.iloc[0]['remover_rows_above_header_column_list'].split(',')
        for index, row in df.iterrows():
            if all(header_value.strip().lower() in [str(x).strip().lower() for x in row.values] for header_value in header_values):
                header_row_index = index
                break

        if header_row_index is not None:
            new_header = df.iloc[header_row_index].apply(lambda x: x.strip().lower())
            df = df.iloc[header_row_index + 1:]
            df.columns = new_header
            df.reset_index(drop=True, inplace=True)
            print("Header set successfully.")
        else:
            print("Error: Suitable header row could not be found based on the provided criteria.")
            continue

        # Applying criteria
        for idx, criteria in criteria_rows.iterrows():
            print(f"Applying criteria: {criteria['applied_column']} {criteria['criteria_to_remove_row']} {criteria['criteria_value']}")
            criteria_dict = {
                'applied_column': criteria['applied_column'].strip().lower(),
                'criteria_to_remove_row': criteria['criteria_to_remove_row'],
                'criteria_value': criteria['criteria_value']
            }
            df = apply_removal_criteria(df, criteria_dict)

        output_file_path.parent.mkdir(parents=True, exist_ok=True)

        # Manage ExcelWriter for output files
        if output_file_path not in excel_writers:
            if output_file_path.exists():
                excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace')
            else:
                excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='w')

        # Save the data to the specified sheet
        df.to_excel(excel_writers[output_file_path], sheet_name=sheet_name, index=False)
        print(f"Data written to {sheet_name} in {output_file_path}")

    # Close all Excel writers
    for path, writer in excel_writers.items():
        writer.close()
        print(f"Closed writer for {path}")


In [11]:
run_removal_automation(file_config_df, cal_df, base_path)

Error: Suitable header row could not be found based on the provided criteria.
Header set successfully.
Applying criteria: year_month = Total
Data written to Sheet2 in C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\output\Test_df_remove.xlsx
Closed writer for C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\output\Test_df_remove.xlsx


In [9]:
def run_removal_automation(file_config_df, cal_df, base_path):
    base_path = Path(base_path)
    excel_writers = {}  # Dictionary to manage ExcelWriter objects for each output file

    for _, file_info in file_config_df.iterrows():
        if pd.isna(file_info['input_folder_path']) or pd.isna(file_info['input_file_name']):
            continue

        input_file_path = base_path / file_info['input_folder_path'] / file_info['input_file_name']
        if not input_file_path.suffix:
            input_file_path = input_file_path.with_suffix(file_info['input_file_type'])

        output_file_path = base_path / file_info['output_folder_path'] / file_info['output_file_name']
        if not output_file_path.suffix:
            output_file_path = output_file_path.with_suffix('.xlsx')

        sheet_name = file_info.get('output_sheet_name', 'Sheet1')

        if not input_file_path.is_file():
            print(f"File not found: {input_file_path}")
            continue

        df = read_input_file_type(input_file_path, file_info['input_file_type'], file_info.get('input_sheet_name'))

        criteria_rows = cal_df[cal_df['linked_input_class'] == file_info['base_input_class']]
        if criteria_rows.empty:
            print(f"No matching criteria found for key: {file_info['base_input_class']}")
            continue

        # Header identification and setting
        header_row_index = None
        header_values = criteria_rows.iloc[0]['remover_rows_above_header_column_list'].split(',')
        
        # Attempt to find the header by searching for rows that contain all header values
        for index, row in df.iterrows():
            if all(header_value.strip().lower() in [str(x).strip().lower() for x in row.values] for header_value in header_values):
                header_row_index = index
                break

        # Check if header is still not found and assume the first row could be the header
        if header_row_index is None:
            if all(df.columns.str.contains('|'.join(header_values), case=False, regex=True)):
                header_row_index = 0

        if header_row_index is not None:
            new_header = df.iloc[header_row_index].apply(lambda x: x.strip().lower())
            df = df.iloc[header_row_index + 1:]
            df.columns = new_header
            df.reset_index(drop=True, inplace=True)
            print("Header set successfully.")
        else:
            print("Error: Suitable header row could not be found based on the provided criteria.")
            continue

        # Applying criteria
        for idx, criteria in criteria_rows.iterrows():
            print(f"Applying criteria: {criteria['applied_column']} {criteria['criteria_to_remove_row']} {criteria['criteria_value']}")
            criteria_dict = {
                'applied_column': criteria['applied_column'].strip().lower(),
                'criteria_to_remove_row': criteria['criteria_to_remove_row'],
                'criteria_value': criteria['criteria_value']
            }
            df = apply_removal_criteria(df, criteria_dict)

        output_file_path.parent.mkdir(parents=True, exist_ok=True)

        # Manage ExcelWriter for output files
        if output_file_path not in excel_writers:
            if output_file_path.exists():
                excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace')
            else:
                excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='w')

        # Save the data to the specified sheet
        df.to_excel(excel_writers[output_file_path], sheet_name=sheet_name, index=False)
        print(f"Data written to {sheet_name} in {output_file_path}")

    # Close all Excel writers
    for path, writer in excel_writers.items():
        writer.close()
        print(f"Closed writer for {path}")

In [10]:
run_removal_automation(file_config_df, cal_df, base_path)

Error: Suitable header row could not be found based on the provided criteria.
Header set successfully.
Applying criteria: year_month = Total
Data written to Sheet2 in C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\output\Test_df_remove.xlsx
Closed writer for C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\output\Test_df_remove.xlsx


In [5]:
import re
def normalize_text(text):
    """ Normalize text to make the comparison more robust. """
    return re.sub(r'[\W_]+', '', text.lower())
    
def run_removal_automation(file_config_df, cal_df, base_path):
    base_path = Path(base_path)
    excel_writers = {}  # Dictionary to manage ExcelWriter objects for each output file

    for _, file_info in file_config_df.iterrows():
        if pd.isna(file_info['input_folder_path']) or pd.isna(file_info['input_file_name']):
            continue

        input_file_path = base_path / file_info['input_folder_path'] / file_info['input_file_name']
        if not input_file_path.suffix:
            input_file_path = input_file_path.with_suffix(file_info['input_file_type'])

        output_file_path = base_path / file_info['output_folder_path'] / file_info['output_file_name']
        if not output_file_path.suffix:
            output_file_path = output_file_path.with_suffix('.xlsx')

        sheet_name = file_info.get('output_sheet_name', 'Sheet1')

        if not input_file_path.is_file():
            print(f"File not found: {input_file_path}")
            continue

        df = read_input_file_type(input_file_path, file_info['input_file_type'], file_info.get('input_sheet_name'))

        criteria_rows = cal_df[cal_df['linked_input_class'] == file_info['base_input_class']]
        if criteria_rows.empty:
            print(f"No matching criteria found for key: {file_info['base_input_class']}")
            continue

        # Header identification and setting
        header_row_index = None
        header_values = [normalize_text(value) for value in criteria_rows.iloc[0]['remover_rows_above_header_column_list'].split(',')]
        
        # First attempt to find the header row by matching normalized content
        for index, row in df.iterrows():
            if all(normalize_text(header_value) in [normalize_text(str(x)) for x in row.values] for header_value in header_values):
                header_row_index = index
                break
        
        # If no header row is found and assuming the first row could be the header
        if header_row_index is None and all(normalize_text(header_value) in [normalize_text(str(x)) for x in df.iloc[0].values] for header_value in header_values):
            header_row_index = 0
        
        if header_row_index is not None:
            new_header = df.iloc[header_row_index].apply(lambda x: x.strip().lower())
            df = df.iloc[header_row_index + 1:] if header_row_index > 0 else df  # Shift data frame down if header is not the first row
            df.columns = new_header
            df.reset_index(drop=True, inplace=True)
            print("Header set successfully.")
        else:
            print("Error: Suitable header row could not be found based on the provided criteria.")
            continue

        # Applying criteria
        for idx, criteria in criteria_rows.iterrows():
            print(f"Applying criteria: {criteria['applied_column']} {criteria['criteria_to_remove_row']} {criteria['criteria_value']}")
            criteria_dict = {
                'applied_column': criteria['applied_column'].strip().lower(),
                'criteria_to_remove_row': criteria['criteria_to_remove_row'],
                'criteria_value': criteria['criteria_value']
            }
            df = apply_removal_criteria(df, criteria_dict)

        output_file_path.parent.mkdir(parents=True, exist_ok=True)

        # Manage ExcelWriter for output files
        if output_file_path not in excel_writers:
            if output_file_path.exists():
                excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace')
            else:
                excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='w')

        # Save the data to the specified sheet
        df.to_excel(excel_writers[output_file_path], sheet_name=sheet_name, index=False)
        print(f"Data written to {sheet_name} in {output_file_path}")

    # Close all Excel writers
    for path, writer in excel_writers.items():
        writer.close()
        print(f"Closed writer for {path}")

In [6]:
run_removal_automation(file_config_df, cal_df, base_path)

Error: Suitable header row could not be found based on the provided criteria.
Header set successfully.
Applying criteria: year_month = Total
Data written to Sheet2 in C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\output\Test_df_remove.xlsx
Closed writer for C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\output\Test_df_remove.xlsx


In [7]:
import re

def normalize_text(text):
    """ Normalize text to make the comparison more robust, including trimming spaces. """
    return re.sub(r'\s+|\W+', '', text).strip().lower()

def run_removal_automation(file_config_df, cal_df, base_path):
    base_path = Path(base_path)
    excel_writers = {}  # Manage ExcelWriter objects for each output file

    for _, file_info in file_config_df.iterrows():
        if pd.isna(file_info['input_folder_path']) or pd.isna(file_info['input_file_name']):
            continue

        input_file_path = base_path / file_info['input_folder_path'] / file_info['input_file_name']
        input_file_path = input_file_path.with_suffix(file_info['input_file_type'] if not input_file_path.suffix else input_file_path.suffix)

        output_file_path = base_path / file_info['output_folder_path'] / file_info['output_file_name']
        output_file_path = output_file_path.with_suffix('.xlsx' if not output_file_path.suffix else output_file_path.suffix)

        sheet_name = file_info.get('output_sheet_name', 'Sheet1')

        if not input_file_path.is_file():
            print(f"File not found: {input_file_path}")
            continue

        df = read_input_file_type(input_file_path, file_info['input_file_type'], file_info.get('input_sheet_name'))

        criteria_rows = cal_df[cal_df['linked_input_class'] == file_info['base_input_class']]
        if criteria_rows.empty:
            print(f"No matching criteria found for key: {file_info['base_input_class']}")
            continue

        header_row_index = None
        header_values = [normalize_text(value) for value in criteria_rows.iloc[0]['remover_rows_above_header_column_list'].split(',')]
        
        for index, row in df.iterrows():
            row_values = [normalize_text(str(x)) for x in row.values]
            if all(header_value in row_values for header_value in header_values):
                header_row_index = index
                print(f"Header found at index: {index}")
                break

        if header_row_index is None:
            first_row_values = [normalize_text(str(x)) for x in df.iloc[0].values]
            if all(header_value in first_row_values for header_value in header_values):
                header_row_index = 0
                print("Assuming first row as header based on criteria match.")

        if header_row_index is not None:
            new_header = df.iloc[header_row_index].apply(lambda x: x.strip().lower())
            df = df.iloc[header_row_index + 1:] if header_row_index > 0 else df
            df.columns = new_header
            df.reset_index(drop=True, inplace=True)
            print("Header set successfully.")
        else:
            print("Error: Suitable header row could not be found based on the provided criteria.")
            continue

        for idx, criteria in criteria_rows.iterrows():
            print(f"Applying criteria: {criteria['applied_column']} {criteria['criteria_to_remove_row']} {criteria['criteria_value']}")
            criteria_dict = {
                'applied_column': criteria['applied_column'].strip().lower(),
                'criteria_to_remove_row': criteria['criteria_to_remove_row'],
                'criteria_value': criteria['criteria_value']
            }
            df = apply_removal_criteria(df, criteria_dict)

        output_file_path.parent.mkdir(parents=True, exist_ok=True)

        if output_file_path not in excel_writers:
            mode = 'a' if output_file_path.exists() else 'w'
            excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode=mode, if_sheet_exists='replace')

        df.to_excel(excel_writers[output_file_path], sheet_name=sheet_name, index=False)
        print(f"Data written to {sheet_name} in {output_file_path}")

    for path, writer in excel_writers.items():
        writer.close()
        print(f"Closed writer for {path}")



In [10]:
run_removal_automation(file_config_df, cal_df, base_path)

Error: Suitable header row could not be found based on the provided criteria.
Header found at index: 6
Header set successfully.
Applying criteria: year_month = Total
Data written to Sheet2 in C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\output\Test_df_remove.xlsx
Closed writer for C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\output\Test_df_remove.xlsx


In [15]:
import re
import pandas as pd
import numpy as np
from pathlib import Path


def read_input_file_type(input_file_path, input_file_type, sheet_name=None):
    if input_file_type == '.xlsx':
        return pd.read_excel(input_file_path, sheet_name=sheet_name, header=None)
    elif input_file_type == '.csv':
        return pd.read_csv(input_file_path, header=None)
    elif input_file_type == '.txt':
        return pd.read_csv(input_file_path, delimiter='\t', header=None)
    else:
        raise ValueError(f"Unsupported file type: {input_file_type}")
        
def apply_removal_criteria(df, criteria):
    
    column = criteria['applied_column']
    if column not in df.columns:
        print(f"The column '{column}' does not exist in the dataframe. Skipping the removal criteria.")
        return df
    
    # column = criteria['applied_column']
    operation = criteria['criteria_to_remove_row']
    value = criteria['criteria_value']
    
    if operation == '=':
        # Remove rows where the column value equals the criteria value
        df = df[df[column].astype(str).str.strip() != value]
    elif operation == '> =':
        # Remove rows where the column value is less than the criteria value
        df = df[df[column] < float(value)]
    elif operation == '>':
        # Remove rows where the column value is less than or equal to the criteria value
        df = df[df[column] <= float(value)]
    elif operation == '< =':
        # Remove rows where the column value is greater than the criteria value
        df = df[df[column] > float(value)]
    elif operation == '<':
        # The '<' case is correctly implemented in the original prompt as removing rows where the value is greater or equal
        df = df[df[column] >= float(value)]  # Correcting based on the initial explanation
    elif operation == 'contain':
        # Remove rows that contain the criteria value
        df = df[~df[column].str.contains(value, na=False, regex=True)]

    return df


def normalize_text(text):
    """ Normalize text to retain dots and commas, converting to lower case for comparison. """
    # Retaining dots, commas, and converting to lower case
    return re.sub(r'[^a-zA-Z0-9.,]+', '', text).lower()

def run_removal_automation(file_config_df, cal_df, base_path):
    base_path = Path(base_path)
    excel_writers = {}  # Manage ExcelWriter objects for each output file

    for _, file_info in file_config_df.iterrows():
        if pd.isna(file_info['input_folder_path']) or pd.isna(file_info['input_file_name']):
            continue

        input_file_path = base_path / file_info['input_folder_path'] / file_info['input_file_name']
        input_file_path = input_file_path.with_suffix(file_info['input_file_type'] if not input_file_path.suffix else input_file_path.suffix)

        output_file_path = base_path / file_info['output_folder_path'] / file_info['output_file_name']
        output_file_path = output_file_path.with_suffix('.xlsx' if not output_file_path.suffix else output_file_path.suffix)

        sheet_name = file_info.get('output_sheet_name', 'Sheet1')

        if not input_file_path.is_file():
            print(f"File not found: {input_file_path}")
            continue

        df = read_input_file_type(input_file_path, file_info['input_file_type'], file_info.get('input_sheet_name'))

        criteria_rows = cal_df[cal_df['linked_input_class'] == file_info['base_input_class']]
        if criteria_rows.empty:
            print(f"No matching criteria found for key: {file_info['base_input_class']}")
            continue

        header_values = [normalize_text(value) for value in criteria_rows.iloc[0]['remover_rows_above_header_column_list'].split(',')]

        # Debug output of first row values
        print("First row values for debugging:", [normalize_text(str(x)) for x in df.iloc[0].values])

        # Directly check the first row as potential header
        first_row_values = [normalize_text(str(x)) for x in df.iloc[0].values]
        if all(header_value in first_row_values for header_value in header_values):
            header_row_index = 0
            print("First row matches header criteria, using as header.")
        else:
            # If not found in the first row, check subsequent rows
            header_row_index = None
            for index, row in df.iterrows():
                row_values = [normalize_text(str(x)) for x in row.values]
                if all(header_value in row_values for header_value in header_values):
                    header_row_index = index
                    print(f"Header found at index: {index}")
                    break

        # if header_row_index is not None:
        #     new_header = df.iloc[header_row_index].apply(lambda x: x.strip().lower())
        #     df = df.iloc[header_row_index + 1:] if header_row_index > 0 else df  # Shift data frame down if header is not the first row
        #     df.columns = new_header
        #     df.reset_index(drop=True, inplace=True)
        #     print("Header set successfully.")
        # else:
        #     print("Error: Suitable header row could not be found based on the provided criteria.")
        #     continue
        
        if header_row_index is not None:
            new_header = df.iloc[header_row_index]  # Assuming this is the correct header
            df = df.iloc[header_row_index + 1:]  # Remove the header row and anything above it
            df.columns = new_header  # Set the new header
            df.reset_index(drop=True, inplace=True)
            print("Header set successfully.")
        else:
            print("Error: Suitable header row could not be found based on the provided criteria.")
    
        for idx, criteria in criteria_rows.iterrows():
            print(f"Applying criteria: {criteria['applied_column']} {criteria['criteria_to_remove_row']} {criteria['criteria_value']}")
            criteria_dict = {
                'applied_column': criteria['applied_column'].strip().lower(),
                'criteria_to_remove_row': criteria['criteria_to_remove_row'],
                'criteria_value': criteria['criteria_value']
            }
            df = apply_removal_criteria(df, criteria_dict)

        output_file_path.parent.mkdir(parents=True, exist_ok=True)

        # Manage ExcelWriter for output files
        if output_file_path not in excel_writers:
            if output_file_path.exists():
                excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace')
            else:
                excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='w')

        # Save the data to the specified sheet
        df.to_excel(excel_writers[output_file_path], sheet_name=sheet_name, index=False)
        print(f"Data written to {sheet_name} in {output_file_path}")

    # Close all Excel writers
    for path, writer in excel_writers.items():
        writer.close()
        print(f"Closed writer for {path}")

In [16]:
run_removal_automation(file_config_df, cal_df, base_path)

First row values for debugging: ['yearmonth', 'yearquarter', 'year6m', 'staffusername', 'position', 'slkhquanly']
First row matches header criteria, using as header.
Header set successfully.
Applying criteria: year_month = nan
Applying criteria: year_month = Applied filters:
Applying criteria: position contain M
Applying criteria: year_month = Total
Data written to Sheet1 in C:\Users\phuc.nguyentuan\Python_project\New Folder\Tien\output\Test_df_remove.xlsx
First row values for debugging: ['a', 'b', 'c', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
Header found at index: 7
Header set successfully.
Applying criteria: year_month = Total
The column 'year_month' does not exist in the dataframe. Skipping the removal criteria.
Data written to Sheet2 in C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\output\Test_df_remove.xlsx
Closed writer for C:\Users\phuc.nguyentuan\Python_project\New Fold

In [7]:
from f01_remove_rows import read_input_file_type, apply_removal_criteria, normalize_text, run_removal_automation

In [8]:
run_removal_automation(file_config_df, cal_df, base_path)

Output file path: C:\Users\phuc.nguyentuan\Python_project\New Folder\Tien\output\Test_df_remove.xlsx
First row values for debugging: ['yearmonth', 'yearquarter', 'year6m', 'staffusername', 'position', 'slkhquanly']
First row matches header criteria, using as header.
Header set successfully.
Applying criteria: nan nan nan
No applied column provided. Skipping this criteria.
Applying criteria: year_month = Applied filters:
Applying criteria: position contain M
Applying criteria: year_month = Total
Excel writer status: True
Data written to Sheet1 in C:\Users\phuc.nguyentuan\Python_project\New Folder\Tien\output\Test_df_remove.xlsx
Output file path: C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\output\Test_df_remove.xlsx
First row values for debugging: ['a', 'b', 'c', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
Header found at index: 7
Header set successfully.
Applying criteria: year_mo

In [11]:
def run_removal_automation(file_config_df, cal_df, base_path):
    base_path = Path(base_path)
    excel_writers = {}  # Manage ExcelWriter objects for each output file

    for _, file_info in file_config_df.iterrows():
        if pd.isna(file_info['input_folder_path']) or pd.isna(file_info['input_file_name']):
            continue

        input_file_path = base_path / file_info['input_folder_path'] / file_info['input_file_name']
        input_file_path = input_file_path.with_suffix(file_info['input_file_type'] if not input_file_path.suffix else input_file_path.suffix)


        output_file_name = file_info['output_file_name']
            # Check if the file name has an extension, if not, append '.xlsx'
        if '.' not in output_file_name:
            output_file_name += '.xlsx'

        output_file_path = base_path / file_info['output_folder_path'] / output_file_name

        print("Output file path:", output_file_path)

        # output_file_path = base_path / file_info['output_folder_path'] / file_info['output_file_name']
        # output_file_path = output_file_path.with_suffix('.xlsx' if not output_file_path.suffix else output_file_path.suffix)

        sheet_name = file_info.get('output_sheet_name', 'Sheet1')

        if not input_file_path.is_file():
            print(f"File not found: {input_file_path}")
            continue

        df = read_input_file_type(input_file_path, file_info['input_file_type'], file_info.get('input_sheet_name'))

        criteria_rows = cal_df[cal_df['linked_input_class'] == file_info['base_input_class']]
        if criteria_rows.empty:
            print(f"No matching criteria found for key: {file_info['base_input_class']}")
            continue

        # header_values = [normalize_text(value) for value in criteria_rows.iloc[0]['remover_rows_above_header_column_list'].split(',')]
        header_values = [normalize_text(value) for value in criteria_rows.iloc[0]['remove_rows_list'].split(',')]

        # Debug output of first row values
        print("First row values for debugging:", [normalize_text(str(x)) for x in df.iloc[0].values])

        # Directly check the first row as potential header
        first_row_values = [normalize_text(str(x)) for x in df.iloc[0].values]
        if all(header_value in first_row_values for header_value in header_values):
            header_row_index = 0
            print("First row matches header criteria, using as header.")
        else:
            # If not found in the first row, check subsequent rows
            header_row_index = None
            for index, row in df.iterrows():
                row_values = [normalize_text(str(x)) for x in row.values]
                if all(header_value in row_values for header_value in header_values):
                    header_row_index = index
                    print(f"Header found at index: {index}")
                    break

        
        if header_row_index is not None:
            new_header = df.iloc[header_row_index]  # Assuming this is the correct header
            df = df.iloc[header_row_index + 1:]  # Remove the header row and anything above it
            # df.columns = [col.strip() for col in new_header]  # Set the new header
            # Strip whitespace and convert to lower case for each header
            df.columns = [col.strip().lower() for col in new_header]  
            df.reset_index(drop=True, inplace=True)
            print("Header set successfully.")
        else:
            print("Error: Suitable header row could not be found based on the provided criteria.")
    
        for idx, criteria in criteria_rows.iterrows():
            print(f"Applying criteria: {criteria['applied_column']} {criteria['criteria_to_remove_row']} {criteria['criteria_value']}")
            criteria_dict = {
                'applied_column': criteria['applied_column'].strip().lower(),
                'criteria_to_remove_row': criteria['criteria_to_remove_row'],
                'criteria_value': criteria['criteria_value']
            }
            df = apply_removal_criteria(df, criteria_dict)

        output_file_path.parent.mkdir(parents=True, exist_ok=True)

        # Manage ExcelWriter for output files
        if output_file_path not in excel_writers:
            if output_file_path.exists():
                excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace')
            else:
                excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='w')
        
        print("Excel writer status:", output_file_path in excel_writers)

        # Save the data to the specified sheet
        df.to_excel(excel_writers[output_file_path], sheet_name=sheet_name, index=False)
        print(f"Data written to {sheet_name} in {output_file_path}")

    # Close all Excel writers
    for path, writer in excel_writers.items():
        writer.close()
        print(f"Closed writer for {path}")

In [12]:
run_removal_automation(file_config_df, cal_df, base_path)

Output file path: C:\Users\phuc.nguyentuan\Python_project\New Folder\Tien\output\Test_df_remove.xlsx
First row values for debugging: ['yearmonth', 'yearquarter', 'year6m', 'staffusername', 'position', 'slkhquanly']
First row matches header criteria, using as header.
Header set successfully.
Applying criteria: year_month = nan
Applying criteria: year_month = Applied filters:
Applying criteria: position contain M
Applying criteria: year_month = Total
Excel writer status: True
Data written to Sheet1 in C:\Users\phuc.nguyentuan\Python_project\New Folder\Tien\output\Test_df_remove.xlsx
Output file path: C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\output\Test_df_remove.xlsx
First row values for debugging: ['a', 'b', 'c', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
Header found at index: 7
Header set successfully.
Applying criteria: year_month = Total
Excel writer status: True
Data writ

In [21]:
print(pd. __version__)

2.1.4


In [36]:
import re
import pandas as pd
import numpy as np
from pathlib import Path

def read_input_file_type(input_file_path, input_file_type, sheet_name=None):
    if input_file_type == '.xlsx':
        return pd.read_excel(input_file_path, sheet_name=sheet_name, header=None)
    elif input_file_type == '.csv':
        return pd.read_csv(input_file_path, header=None)
    elif input_file_type == '.txt':
        return pd.read_csv(input_file_path, delimiter='\t', header=None)
    else:
        raise ValueError(f"Unsupported file type: {input_file_type}")
        
def apply_removal_criteria(df, criteria):
    column = criteria['applied_column']
    if column not in df.columns:
        print(f"The column '{column}' does not exist in the dataframe. Skipping the removal criteria.")
        return df

    operation = criteria['criteria_to_remove_row']
    value = criteria['criteria_value']
    
    if operation == '=':
        df = df[df[column].astype(str).str.strip() != value]
    elif operation == '> =':
        df = df[df[column] < float(value)]
    elif operation == '>':
        df = df[df[column] <= float(value)]
    elif operation == '< =':
        df = df[df[column] > float(value)]
    elif operation == '<':
        df = df[df[column] >= float(value)]
    elif operation == 'contain':
        df = df[~df[column].str.contains(value, na=False, regex=True)]

    return df

def normalize_text(text):
    return re.sub(r'[^a-zA-Z0-9.,]+', '', text).lower()

def run_removal_automation(file_config_df, cal_df, base_path):
    base_path = Path(base_path)
    excel_writers = {}

    for _, file_info in file_config_df.iterrows():
        if pd.isna(file_info['input_folder_path']) or pd.isna(file_info['input_file_name']):
            continue

        input_file_path = base_path / file_info['input_folder_path'] / file_info['input_file_name']
        input_file_path = input_file_path.with_suffix(file_info['input_file_type'] if not input_file_path.suffix else input_file_path.suffix)
        output_file_path = base_path / file_info['output_folder_path'] / file_info['output_file_name']
        output_file_path = output_file_path.with_suffix('.xlsx' if not output_file_path.suffix else output_file_path.suffix)

        sheet_name = file_info.get('output_sheet_name', 'Sheet1')

        if not input_file_path.is_file():
            print(f"File not found: {input_file_path}")
            continue

        df = read_input_file_type(input_file_path, file_info['input_file_type'], file_info.get('input_sheet_name'))
        criteria_rows = cal_df[cal_df['linked_input_class'] == file_info['base_input_class']]

        if criteria_rows.empty:
            print(f"No matching criteria found for key: {file_info['base_input_class']}")
            continue

        if 'remove_rows_above_header_column_list' in criteria_rows.columns and not pd.isna(criteria_rows.iloc[0]['remove_rows_above_header_column_list']):
            header_values = [normalize_text(value) for value in criteria_rows.iloc[0]['remove_rows_above_header_column_list'].split(',')]
            header_row_index = None
            for index, row in df.iterrows():
                row_values = [normalize_text(str(x)) for x in row.values]
                if all(header_value in row_values for header_value in header_values):
                    header_row_index = index
                    print(f"Header found at index: {index}")
                    break

            if header_row_index is not None:
                new_header = df.iloc[header_row_index]
                df = df.iloc[header_row_index + 1:]
                df.columns = new_header
                df.reset_index(drop=True, inplace=True)
                print("Header set successfully.")
            else:
                print("Warning: Suitable header row could not be found based on the provided criteria. Defaulting to first row as header.")
                df.columns = df.iloc[0]
                df = df.iloc[1:]
                df.reset_index(drop=True, inplace=True)
        else:
            print("Header removal criteria missing or contains NaN. Defaulting to first row as header.")
            df.columns = df.iloc[0]
            df = df.iloc[1:]
            df.reset_index(drop=True, inplace=True)

        for idx, criteria in criteria_rows.iterrows():
            print(f"Applying criteria: {criteria['applied_column']} {criteria['criteria_to_remove_row']} {criteria['criteria_value']}")
            criteria_dict = {
                'applied_column': criteria['applied_column'].strip().lower(),
                'criteria_to_remove_row': criteria['criteria_to_remove_row'],
                'criteria_value': criteria['criteria_value']
            }
            df = apply_removal_criteria(df, criteria_dict)

        output_file_path.parent.mkdir(parents=True, exist_ok=True)
        if output_file_path not in excel_writers:
            if output_file_path.exists():
                excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace')
            else:
                excel_writers[output_file_path] = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='w')

        df.to_excel(excel_writers[output_file_path], sheet_name=sheet_name, index=False)
        print(f"Data written to {sheet_name} in {output_file_path}")

    for path, writer in excel_writers.items():
        writer.close()
        print(f"Closed writer for {path}")

In [37]:
run_removal_automation(file_config_df, cal_df, base_path)

Header removal criteria missing or contains NaN. Defaulting to first row as header.
Applying criteria: year_month = nan
Applying criteria: year_month = Applied filters:
Applying criteria: position contain M
Applying criteria: year_month = Total
Data written to Sheet1 in C:\Users\phuc.nguyentuan\Python_project\New Folder\Tien\output\Test_df_remove.xlsx
Header removal criteria missing or contains NaN. Defaulting to first row as header.
Applying criteria: year_month = Total
The column 'year_month' does not exist in the dataframe. Skipping the removal criteria.
Data written to Sheet2 in C:\Users\phuc.nguyentuan\Python_project\New folder\Tien\output\Test_df_remove.xlsx
Closed writer for C:\Users\phuc.nguyentuan\Python_project\New Folder\Tien\output\Test_df_remove.xlsx
